<a href="https://colab.research.google.com/github/AlexKressner/Vorlesung-Industrielles-Mangement/blob/main/NetworkDesing_Daten_von_csv_laden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Design: Daten mit `pandas` aus csv-Dateien laden

In [ ]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

In [ ]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Daten aus csv-Datei laden


1.   Problemrelevante Daten in Google-Drive laden
2.   Google-Drive mit Colab-Notebook verbinden
3.   Daten mit `pandas` laden



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Ordner finden
! ls drive/MyDrive/Industrielles_Management/Daten

NetworkDesign_Invest_Kapa.csv  NetworkDesign_Produktionskosten.csv
NetworkDesign_Nachfrage.csv


In [ ]:
# Pfad zurückgeben
! cd drive/MyDrive/Industrielles_Management/Daten && pwd

/content/drive/MyDrive/Industrielles_Management/Daten


In [ ]:
# Daten laden
import pandas as pd

In [ ]:
path = "/content/drive/MyDrive/Industrielles_Management/Daten"

In [ ]:
kapa=pd.read_csv(f"{path}/NetworkDesign_Invest_Kapa.csv", sep=";")

In [ ]:
kapa.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [ ]:
produktionskosten=pd.read_csv(f"{path}/NetworkDesign_Produktionskosten.csv", sep=";")

In [ ]:
produktionskosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [ ]:
nachfrage=pd.read_csv(f"{path}/NetworkDesign_Nachfrage.csv", sep=";")

In [ ]:
nachfrage.head()

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


## Indexmengen

In [ ]:
I = produktionskosten["Produktionsstandort"].unique().tolist() # Menge der Produktionsstandorte

In [ ]:
J = nachfrage["Markt"].tolist() #Menge der Märkte

In [ ]:
A = kapa["Ausbaustufe"].unique().tolist() # Menge der Ausbaustufen an einem Standort

## Entscheidungsvariablen

In [ ]:
infinity = solver.infinity()

In [ ]:
# Definition der Entscheidungsvariablen
# x sind die Flussvariablen, d.h. Herstell- und Transportmenge von i nach j
x={}
for i in I:
  for j in J: 
    x[i,j] = solver.NumVar(0.0, infinity, f"{i},{j}")

In [ ]:
# Definition der Entscheidungsvariablen
# y sind die Strukturvariablen, d.h. ob und mit welchen Kapazitäten ein Standort ausgebaut wird
y={}
for i in I:
  for a in A: 
    y[i,a] = solver.BoolVar(f"{i},{a}")

In [ ]:
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 35


## Parameter

In [ ]:
# Herstell- und Transportkosten
produktionskosten.set_index(["Produktionsstandort","Markt"], inplace=True)

In [ ]:
c = produktionskosten.to_dict("dict")["Produktionskosten"]

In [ ]:
c

{('Charleston', 'USA'): 81,
 ('Charleston', 'Südamerika'): 92,
 ('Charleston', 'Europa'): 101,
 ('Charleston', 'Asien'): 130,
 ('Charleston', 'Afrika'): 115,
 ('Curitiba', 'USA'): 117,
 ('Curitiba', 'Südamerika'): 77,
 ('Curitiba', 'Europa'): 108,
 ('Curitiba', 'Asien'): 98,
 ('Curitiba', 'Afrika'): 100,
 ('Hamburg', 'USA'): 102,
 ('Hamburg', 'Südamerika'): 105,
 ('Hamburg', 'Europa'): 95,
 ('Hamburg', 'Asien'): 119,
 ('Hamburg', 'Afrika'): 111,
 ('Quanzhou', 'USA'): 115,
 ('Quanzhou', 'Südamerika'): 125,
 ('Quanzhou', 'Europa'): 90,
 ('Quanzhou', 'Asien'): 59,
 ('Quanzhou', 'Afrika'): 74,
 ('East London', 'USA'): 142,
 ('East London', 'Südamerika'): 100,
 ('East London', 'Europa'): 103,
 ('East London', 'Asien'): 105,
 ('East London', 'Afrika'): 71}

In [ ]:
# Nachfrage aus den Märtken

In [ ]:
nachfrage.set_index(["Markt"], inplace=True)

In [ ]:
d = nachfrage.to_dict("dict")["Nachfragemenge"]

In [ ]:
d

{'USA': 12, 'Südamerika': 8, 'Europa': 14, 'Asien': 16, 'Afrika': 7}

In [ ]:
# Kapazitäten der Ausbaustufen und Investitionskosten
kapa.set_index(["Produktionsstandort","Ausbaustufe"], inplace=True)

In [ ]:
cap = kapa.to_dict("dict")["Kapazitäten"]

In [ ]:
cap

{('Charleston', 'klein'): 10,
 ('Charleston', 'groß'): 20,
 ('Curitiba', 'klein'): 10,
 ('Curitiba', 'groß'): 20,
 ('Hamburg', 'klein'): 10,
 ('Hamburg', 'groß'): 20,
 ('Quanzhou', 'klein'): 10,
 ('Quanzhou', 'groß'): 20,
 ('East London', 'klein'): 10,
 ('East London', 'groß'): 20}

In [ ]:
f = kapa.to_dict("dict")["Investitionen"]

In [ ]:
f

{('Charleston', 'klein'): 6000,
 ('Charleston', 'groß'): 9000,
 ('Curitiba', 'klein'): 4500,
 ('Curitiba', 'groß'): 6750,
 ('Hamburg', 'klein'): 6500,
 ('Hamburg', 'groß'): 9750,
 ('Quanzhou', 'klein'): 4100,
 ('Quanzhou', 'groß'): 6150,
 ('East London', 'klein'): 4000,
 ('East London', 'groß'): 6000}

## Zielfunktion

## Nebenbedingungen

## Berechnung Lösung